# Data

In [230]:
data = {
    'start' : '2020/03/02',
    'cases' : [
        2,
        4,
        6,
        9,
        13,
        21,
        30,
        39,
        41,
        59,
        78,
        112,
        169,
        245,
        331,
        448,
        642,
        785,
        1020,
        1280,
        1600,
        2060,
        2362,
        2995,
        3544,
        4268
    ]
}

# Models

In [231]:
from numpy import exp

# exponetial function
def exponential_model(x, a, b, c):
    return a * exp( b * (x - c))

# logistic function
def logistic_model(x, a, b, c):
    return c / (1 + exp(-(x - b) / a))

# Regression 

In [232]:
from scipy.optimize import curve_fit, fsolve
# prepare data
y = data['cases']
x = range(len(y))

# exponential fit
exponent_fit = curve_fit(exponential_model, x, y)
# exponential parameters
exp_a = exponent_fit[0][0]
exp_b = exponent_fit[0][1]
exp_c = exponent_fit[0][2]
# logistic fit
logistic_fit = curve_fit(logistic_model, x, y)
# logistic parameters
log_a = logistic_fit[0][0]
log_b = logistic_fit[0][1]
log_c = logistic_fit[0][2]

# logistic end day
end = int(fsolve(lambda x : logistic_model(x, log_a, log_b, log_c) - int(log_c), log_b))

[2, 4, 6, 9, 13, 21, 30, 39, 41, 59, 78, 112, 169, 245, 331, 448, 642, 785, 1020, 1280, 1600, 2060, 2362, 2995, 3544, 4268]


ValueError: operands could not be broadcast together with shapes (2,) (26,) 

# Results

In [233]:
import matplotlib.pyplot as plt

# figure
plt.figure('graph', figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

# Limits
pad = 0.1 * log_c
plt.ylim((min(y) - pad, log_c + pad))

# Real data
plt.scatter(x, y, label="Real data", color="red")

# Predicted exponetial curve
x = range(end)
plt.plot(x, [exponential_model(i, exp_a, exp_b, exp_c) for i in x], label="Exponential model")

# Predicted logistic curve
x = range(end)
plt.plot(x, [logistic_model(i, log_a, log_b, log_c) for i in x], label="Logistic model")

# show
plt.show()

ValueError: x and y must be the same size

In [234]:
from IPython.display import Markdown, display
from datetime        import datetime, date, timedelta
# start date
start_date = datetime.strptime(data['start'], '%Y/%m/%d').date()
# conclusion
display(Markdown(f'# End of epidemic {start_date + timedelta(days=end)} :smile:'))


# End of epidemic 2020-04-30 :smile: